In [13]:
from dotenv import load_dotenv
import os
import time
import mysql.connector
import requests
import mysql.connector
import json
import datetime
import pandas as pd

In [14]:
# Your GitHub personal access token 
# Set request header information, including authorization token
load_dotenv()
github_token = os.getenv("GITHUB_TOKEN")
headers = {'Authorization': 'Bearer ' + github_token}

In [7]:
def connect_to_mysql(database):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="123456",
        database=database
    )
    return conn

# Retrieve data from MySQL
def fetch_data_from_mysql(conn, table_name, *columns):
    columns_str = ', '.join(columns)
    query = "SELECT {} FROM {}".format(columns_str, table_name)
    df = pd.read_sql(query, conn)
    return df

In [10]:
# Retrieve the creator, creation time, closure time, and number of comments for the issue
def get_github_issues(owner, repo, id_list, headers, tableName):
    j = 0
    while j < len(id_list):
        i = j
        j = j + 200
        all_issues = []
        # Loop through each submitted ID
        while i < j and i < len(id_list):
            url = f"https://api.github.com/repos/{owner}/{repo}/issues/{id_list[i]}"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                issue = response.json()
                all_issues.append(issue)
            else:
                print(f"Failed to retrieve issue information for id {id_list[i]}. Status code:", response.status_code)
            i = i + 1
        time.sleep(3)
#         print('Now the i-value is: ', i)

        conn = connect_to_mysql('deep_learning')
        cursor = conn.cursor()
        for issue_info in all_issues:
            issueId = issue_info['number']
            creator = issue_info['user']['login']
            createdAt = datetime.datetime.strptime(issue_info['created_at'], '%Y-%m-%dT%H:%M:%SZ') if issue_info['created_at'] else None
            closedAt = datetime.datetime.strptime(issue_info['closed_at'], '%Y-%m-%dT%H:%M:%SZ') if issue_info['closed_at'] else None
            commentCount = issue_info['comments']

            insert_query = f"UPDATE {tableName} SET IssueCreator = %s, IssueCreatedAt = %s, IssueClosedAt = %s, IssueCommentsNum = %s WHERE IssueID = %s"
            try:
                cursor.execute(insert_query, (creator, createdAt, closedAt, commentCount, issueId))
                conn.commit()
            except:
                print("Insert error"+ str(i) + " issueid:" + str(issueId))
                conn.rollback()
        cursor.close()
        conn.close()

In [11]:
# Retrieve comments on the issue and return JSON data
def get_github_issues_comments(owner, repo, id_list, headers, tableName):
    j = 0
    while j < len(id_list):
        i = j
        j = j + 200
        all_comments = []
        while i < j and i < len(id_list):
            url = f"https://api.github.com/repos/{owner}/{repo}/issues/{id_list[i]}/comments"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                comment = response.json()
                all_comments.append(comment)
            else:
                print(f"Failed to retrieve issue comments information for id {id_list[i]}. Status code:", response.status_code)
            i = i + 1
        time.sleep(3)
#         print('Now the i-value is: ', i)

        conn = connect_to_mysql('deep_learning')
        cursor = conn.cursor()
        for issue_info in all_comments:
            if not issue_info:
                continue
            issueId = issue_info[0]['issue_url'].split('/')[-1]
            comments = json.dumps([{'id': dic['id'], 'url': dic['html_url'], 'creator': dic['user']['login'], 'createdAt': dic['created_at'], 'body': dic['body']} for dic in issue_info])

            insert_query = f"UPDATE {tableName} SET IssueComments = %s WHERE IssueID = %s"
            try:
                cursor.execute(insert_query, (comments, issueId))
                conn.commit()
            except:
                print("Insert error"+ str(i) + " issueid:" + str(issueId))
                conn.rollback()
        cursor.close()
        conn.close()

In [15]:
# Get issue comments for MXNet
owner = 'apache'
repo = 'mxnet'

conn = connect_to_mysql('deep_learning')
df_issues_mx = fetch_data_from_mysql(conn, 'issue_content_mx', 'IssueID')
conn.close()
# print('Number of data entries: ', df_issues_mx.shape[0])

id_list_mx = df_issues_mx['IssueID'].tolist()

get_apache_issues(owner, repo, id_list_mx, headers, 'issue_content_mx')
get_apache_issues_comments(owner, repo, id_list_mx, headers, 'issue_content_mx')

In [16]:
# Get issue comments for pytorch
owner = 'pytorch'
repo = 'pytorch'

conn = connect_to_mysql('deep_learning')
df_issues_py = fetch_data_from_mysql(conn, 'issue_content_py', 'IssueID')
conn.close()
# print('Number of data entries: ', df_issues_py.shape[0])

id_list_py = df_issues_py['IssueID'].tolist()

get_github_issues(owner, repo, id_list_py, headers, 'issue_content_py')
get_github_issues_comments(owner, repo, id_list_py, headers, 'issue_content_py')

In [17]:
# Get issue comments for tensorflow
owner = 'tensorflow'
repo = 'tensorflow'

conn = connect_to_mysql('deep_learning')
df_issues_tf = fetch_data_from_mysql(conn, 'issue_content_tf', 'IssueID')
conn.close()
# print('Number of data entries: ', df_issues_tf.shape[0])

id_list_tf = df_issues_tf['IssueID'].tolist()

get_apache_issues(owner, repo, id_list_tf, headers, 'issue_content_tf')
get_apache_issues_comments(owner, repo, id_list_tf, headers, 'issue_content_tf')

In [18]:
# Retrieve the creator, creation time, closure time, and number of comments for the pull request
def get_apache_pulls(owner, repo, id_list, headers, tableName):
    j = 0
    while j < len(id_list):
        i = j
        j = j + 200
        all_pulls = []
        while i < j and i < len(id_list):
            url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{id_list[i]}"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                pull = response.json()
                all_pulls.append(pull)
            else:
                print(f"Failed to retrieve pull information for id {id_list[i]}. Status code:", response.status_code)
            i = i + 1
        time.sleep(3)
#         print('At this point, the value of i is:', i)

        conn = connect_to_mysql('deep_learning')
        cursor = conn.cursor()
        for pull_info in all_pulls:
            prId = pull_info['number']
            creator = pull_info['user']['login']
            createdAt = datetime.datetime.strptime(pull_info['created_at'], '%Y-%m-%dT%H:%M:%SZ') if pull_info['created_at'] else None
            closedAt = datetime.datetime.strptime(pull_info['closed_at'], '%Y-%m-%dT%H:%M:%SZ') if pull_info['closed_at'] else None
            commentCount = pull_info['review_comments']

            insert_query = f"UPDATE {tableName} SET PrCreator = %s, PrCreatedAt = %s, PrClosedAt = %s, PrCommentsNum = %s WHERE PrID = %s"
            try:
                cursor.execute(insert_query, (creator, createdAt, closedAt, commentCount, prId))
                conn.commit()
            except:
                print("insert error"+ str(i) + " prid:" + str(prId))
                conn.rollback()
        cursor.close()
        conn.close()

In [19]:
# Retrieve comments on the pull request and return JSON data
def get_apache_pulls_comments(owner, repo, id_list, headers, tableName):
    j = 0
    while j < len(id_list):
        i = j
        j = j + 200
        all_comments = []
        while i < j and i < len(id_list):
            url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{id_list[i]}/comments"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                comment = response.json()
                all_comments.append(comment)
            else:
                print(f"Failed to retrieve pull comments information for id {id_list[i]}. Status code:", response.status_code)
            i = i + 1
        time.sleep(3)
#         print('At this point, the value of i is:', i)

        conn = connect_to_mysql('deep_learning')
        cursor = conn.cursor()
        for pr_info in all_comments:
            if not pr_info:
                continue
            prId = pr_info[0]['pull_request_url'].split('/')[-1]
            comments = json.dumps([{'id': dic['id'], 'url': dic['html_url'], 'creator': dic['user']['login'], 'createdAt': dic['created_at'], 'body': dic['body']} for dic in pr_info])

            insert_query = f"UPDATE {tableName} SET PrComments = %s WHERE PrID = %s"
            try:
                cursor.execute(insert_query, (comments, prId))
                conn.commit()
            except:
                print("insert error"+ str(i) + " prid:" + str(prId))
                conn.rollback()
        cursor.close()
        conn.close()

In [20]:
# Get pull request comments for mxnet
owner = 'apache'
repo = 'mxnet'

conn = connect_to_mysql('deep_learning')
df_pull_mx = fetch_data_from_mysql(conn, 'pr_content_mx', 'PrID')
conn.close()
# print('Number of data entries: ', df_pull_mx.shape[0])

id_list_mx = df_pull_mx['PrID'].tolist()

get_apache_pulls(owner, repo, id_list_mx, headers, 'pr_content_mx')
get_apache_pulls_comments(owner, repo, id_list_mx, headers, 'pr_content_mx')

In [21]:
# Get pull request comments for pytorch
owner = 'pytorch'
repo = 'pytorch'

conn = connect_to_mysql('deep_learning')
df_pull_py = fetch_data_from_mysql(conn, 'pr_content_py', 'PrID')
conn.close()
# print('Number of data entries: ', df_pull_py.shape[0])

id_list_py = df_pull_py['PrID'].tolist()

get_apache_pulls(owner, repo, id_list_py, headers, 'pr_content_py')
get_apache_pulls_comments(owner, repo, id_list_py, headers, 'pr_content_py')

In [22]:
# Get pull request comments for tensorflow
owner = 'tensorflow'
repo = 'tensorflow'

conn = connect_to_mysql('deep_learning')
df_pull_tf = fetch_data_from_mysql(conn, 'pr_content_tf', 'PrID')
conn.close()
# print('Number of data entries: ', df_pull_tf.shape[0])

id_list_tf = df_pull_tf['PrID'].tolist()

get_apache_pulls(owner, repo, id_list_tf, headers, 'pr_content_tf')
get_apache_pulls_comments(owner, repo, id_list_tf, headers, 'pr_content_tf')